In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper

# USER DATA

In [ ]:
path_users = "/Users/gandalf/Documents/data/data_users.pkl"
user_df = pd.read_pickle(path_users)

# REMOVE NEW WORLD
not_NA_user_df = user_df[(user_df.longitude > -25) | (user_df.latitude < 0)]
not_NA_user_df = not_NA_user_df[user_df.longitude != 0]

not_NA_user_df.head(2)

In [ ]:
outside_NA_uid = not_NA_user_df.index
print(len(outside_NA_uid))

In [ ]:
path_message = "/Users/gandalf/Documents/data/data_message.pkl"
message_df = pd.read_pickle(path_message)
message_df.head(2)

In [ ]:
print("All messages: {}".format(len(message_df)))
outside_NA_messages = message_df[message_df.uid.isin(outside_NA_uid)]
print("Outside NA messages: {}".format(len(outside_NA_messages)))

In [ ]:
outside_NA.head()

In [ ]:
path_lastmessage = "/Users/gandalf/Documents/data/data_lastmessage.pkl"
lastmessage_df = pd.read_pickle(path_lastmessage)
lastmessage_df.head(2)

In [ ]:
path_response = "/Users/gandalf/Documents/data/data_response.pkl"
response_df = pd.read_pickle(path_response)
response_df.head(5)

In [ ]:
print("All conversations: {}".format(len(response_df)))
outside_NA_conv = response_df[response_df.first_uid.isin(outside_NA_uid)]
print("Outside NA conversations: {}".format(len(outside_NA_conv)))
outside_NA_conv.head()

In [ ]:
outside_NA_conv.response.sum()/337

In [ ]:
outside_NA_conv['first_ten'] = outside_NA_conv.conv_id.apply(lambda x: x[:10])
outside_NA_conv['last_ten'] = outside_NA_conv.conv_id.apply(lambda x: x[10:])

In [ ]:
outside_NA_conv.head()

In [ ]:
# get recipients
for index, row in outside_NA_conv.iterrows():
    if row.second_uid == None:
        if row.first_ten == row.first_uid: row.second_uid = row.last_ten
        elif row.last_ten == row.first_uid: row.second_uid = row.first_ten
        else: print('uh')

In [ ]:
outside_NA_conv[outside_NA_conv.response].head()

In [ ]:
simple_cols = ['first_uid','second_uid','response']
simple_df = outside_NA_conv[simple_cols]
# tf = {True:1, False:0, 1:1, 0:0}
# simple_df.response = simple_df.response.map(tf)
simple_df.head()

In [ ]:
print(simple_df.response.sum())
print(len(simple_df.response))

In [ ]:
len(user_df)

In [ ]:
user_df.loc['0cP281Y8eL'].isClean

In [ ]:
simple_df.head(1)

In [ ]:
ad,sg,sr,sc,sn,ss = [],[],[],[],[],[]

for index, row in simple_df.iterrows():
    first = user_df.loc[str(row.first_uid)]
    second = user_df.loc[str(row.second_uid)]
    ad.append(abs(first.age - second.age))
    sg.append(first.gender == second.gender)
    sr.append(first.inRelationship == second.inRelationship)
    sc.append(first.isClean == second.isClean)
    sn.append(first.isNight == second.isNight)
    ss.append(first.isStudent == second.isStudent)

simple_df['age_dif'] = ad
simple_df['same_gender'] = sg
simple_df['same_relate'] = sr
simple_df['same_clean'] = sc
simple_df['same_night'] = sn
simple_df['same_student'] = ss

# filled na at some point, should undo

simple_df.head(2)

In [ ]:
X = simple_df
y = X.response.map({True:1,False:0})
X = X.drop(['first_uid','second_uid','response'], axis=1)
X.head()
y.sum()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=17)

mapper = DataFrameMapper([(X_train.columns, StandardScaler())])
scale = mapper.fit(X_train)
X_train = pd.DataFrame(mapper.transform(X_train), index=X_train.index, columns=X_train.columns)
X_test = pd.DataFrame(mapper.transform(X_test), index=X_test.index, columns=X_test.columns)

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

print("Score of model: {}".format(model.score(X_test, y_test)))

print("Score to beat: {}".format(1-y_test.sum()/len(y_test)))

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

print("Score of model: {}".format(model.score(X_test, y_test)))

print("Score to beat: {}".format(1-y_test.sum()/len(y_test)))